In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model, Sequential
from keras.layers import (
    Dense,
    Flatten,
    Dropout,
    Input,
)
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.preprocessing import LabelBinarizer
from sklearn.metrics import (
    accuracy_score,
    precision_score,
    recall_score,
    confusion_matrix,
    f1_score,
)
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import glob as gb
import cv2
from tqdm import tqdm
from collections import Counter
from sklearn.model_selection import train_test_split
from keras.applications import resnet

In [2]:
shape = (150, 150, 3)
image_size = 150
path = "./Images/"

In [3]:
def read_images_from_directory(
    folder_path,
    desc,
):
    images = []
    labels = []

    for folder in os.listdir(folder_path):
        files = gb.glob(pathname=str(folder_path + "/" + folder + "/*.jpg"))

        for file in tqdm(files, desc=desc + " in " + folder):
            image = plt.imread(file)
            img_resized = cv2.resize(image, (image_size, image_size))
            img_rgb = cv2.cvtColor(img_resized, cv2.COLOR_BGR2RGB)
            images.append(img_rgb)
            labels.append(folder)

    return images, labels

In [4]:
x, y = read_images_from_directory(path, "Loading Training Data")

Loading Training Data in pituitary_tumor: 100%|██████████| 4504/4504 [00:44<00:00, 100.17it/s]


In [5]:
x_data = np.array(x)
len(x_data)

18818

In [6]:
lb = LabelBinarizer()
y_data = lb.fit_transform(y)

In [7]:
from sklearn.model_selection import train_test_split

x_temp, x_test, y_temp, y_test = train_test_split(
    x_data, y_data, test_size=0.2, random_state=42
)

In [8]:
from sklearn.model_selection import KFold

kf = KFold(n_splits=5, shuffle=True, random_state=42)
fold_no = 1
acc_per_fold = []
loss_per_fold = []

In [9]:
def create_model():
    model = Sequential()
    model.add(Input(shape=shape))

    base_model = resnet.ResNet50(
        include_top=False, input_shape=shape, weights="imagenet"
    )

    for layer in base_model.layers:
        layer.trainable = False

    model.add(base_model)
    model.add(Flatten())
    model.add(Dense(512, activation="relu"))
    model.add(Dropout(0.3))
    model.add(Dense(4, activation="softmax"))

    model.compile(
        optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"]
    )
    # model.summary()
    return model

In [10]:
model = create_model()
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50 (Functional)       (None, 5, 5, 2048)        23587712  
                                                                 
 flatten (Flatten)           (None, 51200)             0         
                                                                 
 dense (Dense)               (None, 512)               26214912  
                                                                 
 dropout (Dropout)           (None, 512)               0         
                                                                 
 dense_1 (Dense)             (None, 4)                 2052      
                                                                 
Total params: 49,804,676
Trainable params: 26,216,964
Non-trainable params: 23,587,712
_________________________________________________________________


In [11]:
from tensorflow.keras.callbacks import EarlyStopping

earlystop = EarlyStopping(
    monitor="val_loss", min_delta=0, patience=10, verbose=1, restore_best_weights=True
)

In [12]:
for train_index, val_index in kf.split(x_temp):
    x_train_fold, x_val_fold = x_temp[train_index], x_temp[val_index]
    y_train_fold, y_val_fold = y_temp[train_index], y_temp[val_index]

    resnetss01 = create_model()

    print(f"Training fold {fold_no}...")
    history01 = resnetss01.fit(
        x_train_fold,
        y_train_fold,
        batch_size=32,
        validation_data=(x_val_fold, y_val_fold),
        epochs=50,
        callbacks=earlystop,
    )

    scores = resnetss01.evaluate(x_val_fold, y_val_fold, verbose=0)
    print(
        f"Score for fold {fold_no}: {resnetss01.metrics_names[0]} of {scores[0]}; {resnetss01.metrics_names[1]} of {scores[1]*100}%"
    )
    acc_per_fold.append(scores[1] * 100)
    loss_per_fold.append(scores[0])

    fold_no += 1

    print(acc_per_fold)
    print(loss_per_fold)

Training fold 1...
Epoch 1/50
377/377 [==============================] - 43s 85ms/step - loss: 2.2625 - accuracy: 0.7583 - val_loss: 0.3544 - val_accuracy: 0.8579
Epoch 2/50
377/377 [==============================] - 30s 79ms/step - loss: 0.3595 - accuracy: 0.8636 - val_loss: 0.2957 - val_accuracy: 0.8907
Epoch 3/50
377/377 [==============================] - 29s 78ms/step - loss: 0.2654 - accuracy: 0.8950 - val_loss: 0.2285 - val_accuracy: 0.9153
Epoch 4/50
377/377 [==============================] - 29s 78ms/step - loss: 0.2083 - accuracy: 0.9196 - val_loss: 0.2507 - val_accuracy: 0.9150
Epoch 5/50
377/377 [==============================] - 29s 78ms/step - loss: 0.1971 - accuracy: 0.9264 - val_loss: 0.2303 - val_accuracy: 0.9263
Epoch 6/50
377/377 [==============================] - 30s 78ms/step - loss: 0.1610 - accuracy: 0.9376 - val_loss: 0.3241 - val_accuracy: 0.8970
Epoch 7/50
377/377 [==============================] - 30s 79ms/step - loss: 0.1535 - accuracy: 0.9444 - val_loss: 0.2

In [13]:
# Print overall results
print("Average scores for all folds:")
print(f"> Accuracy: {np.average(acc_per_fold)}")
print(f"> Loss: {np.average(loss_per_fold)}")

print("Scores per fold:")
for i in range(len(acc_per_fold)):
    print(f"> Fold {i+1} - Loss: {loss_per_fold[i]} - Accuracy: {acc_per_fold[i]}%")

Average scores for all folds:
> Accuracy: 92.81922459602356
> Loss: 0.21930547654628754
Scores per fold:
> Fold 1 - Loss: 0.2260897308588028 - Accuracy: 93.82265210151672%
> Fold 2 - Loss: 0.22029206156730652 - Accuracy: 92.06243753433228%
> Fold 3 - Loss: 0.23425929248332977 - Accuracy: 91.43142104148865%
> Fold 4 - Loss: 0.21014128625392914 - Accuracy: 93.35768818855286%
> Fold 5 - Loss: 0.20574501156806946 - Accuracy: 93.4219241142273%


In [14]:
import numpy as np


def plot_training_curves(history):
    loss = np.array(history.history["loss"])
    val_loss = np.array(history.history["val_loss"])

    accuracy = np.array(history.history["accuracy"])
    val_accuracy = np.array(history.history["val_accuracy"])

    epochs = range(len(history.history["loss"]))

    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 6))

    # plot loss
    ax1.plot(epochs, loss, label="traing_loss", marker="o")

    ax1.plot(epochs, val_loss, label="val_loss", marker="o")

    ax1.fill_between(
        epochs,
        loss,
        val_loss,
        where=(loss > val_loss),
        color="C0",
        alpha=0.3,
        interpolate=True,
    )
    ax1.fill_between(
        epochs,
        loss,
        val_loss,
        where=(loss < val_loss),
        color="C1",
        alpha=0.3,
        interpolate=True,
    )

    ax1.set_title("Loss", fontsize=16)
    ax1.set_xlabel("Epochs", fontsize=10)

    ax1.legend()

    # plot Accuracy
    ax2.plot(epochs, accuracy, label="traing_accuracy", marker="o")

    ax2.plot(epochs, val_accuracy, label="val_accuracy", marker="o")

    ax2.fill_between(
        epochs,
        accuracy,
        val_accuracy,
        where=(accuracy > val_accuracy),
        color="C0",
        alpha=0.3,
        interpolate=True,
    )
    ax2.fill_between(
        epochs,
        accuracy,
        val_accuracy,
        where=(accuracy < val_accuracy),
        color="C1",
        alpha=0.3,
        interpolate=True,
    )

    ax2.set_title("Accuracy", fontsize=16)
    ax2.set_xlabel("Epochs", fontsize=10)

    ax2.legend()

In [15]:
plot_training_curves(history01)

In [16]:
test_loss, test_acc = resnetss01.evaluate(x_data, y_data)
print(f"Test accuracy: {test_acc}")
print(f"Test loss: {test_loss}")

InternalError: Failed copying input tensor from /job:localhost/replica:0/task:0/device:CPU:0 to /job:localhost/replica:0/task:0/device:GPU:0 in order to run _EagerConst: Dst tensor is not initialized.

In [17]:
y_pred_probs = resnetss01.predict(x_test)
y_pred = np.argmax(y_pred_probs, axis=1)
y_true = np.argmax(y_test, axis=1)

118/118 [==============================] - 8s 59ms/step


In [18]:
conf_matrix = confusion_matrix(y_true, y_pred)

accuracy = accuracy_score(y_true, y_pred)

precision = precision_score(y_true, y_pred, average="weighted")

recall = recall_score(y_true, y_pred, average="weighted")

f1 = f1_score(y_true, y_pred, average="weighted")

In [19]:
print(f"Accuracy: {accuracy}")

print(f"Precision: {precision}")

print(f"Recall: {recall}")

print(f"F1 Score: {f1}")

Confusion Matrix:
[[844  58  14   2]
 [ 83 826  14  38]
 [  6  12 974   4]
 [  7  50   4 828]]
Accuracy: 0.922422954303932
Precision: 0.9223709759251904
Recall: 0.922422954303932
F1 Score: 0.9223250864163745


In [20]:
plt.figure(figsize=(10, 8))


sns.heatmap(
    conf_matrix,
    annot=True,
    fmt="d",
    cmap="Blues",
    xticklabels=lb.classes_,
    yticklabels=lb.classes_,
)


plt.xlabel("Predicted")


plt.ylabel("Actual")


plt.title("Confusion Matrix")


plt.show()

In [21]:
model_directory = "Models"
if not os.path.exists(model_directory):
    os.makedirs(model_directory)
model_path = os.path.join(model_directory, "ResNet50_AUG_K_Fold_01(150x150).h5")
resnetss01.save(model_path)
print(f"Model saved to {model_path}")

Model saved to Models\ResNet50_AUG_K_Fold_01(150x150).h5


In [7]:
def create_model():
    model = Sequential()
    model.add(Input(shape=shape))

    base_model = resnet.ResNet50(
        include_top=False, input_shape=shape, weights="imagenet"
    )

    model.add(base_model)
    model.add(Flatten())
    model.add(Dense(512, activation="relu"))
    model.add(Dropout(0.3))
    model.add(Dense(4, activation="softmax"))

    model.compile(
        optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"]
    )
    return model

In [8]:
model = create_model()
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50 (Functional)       (None, 5, 5, 2048)        23587712  
                                                                 
 flatten (Flatten)           (None, 51200)             0         
                                                                 
 dense (Dense)               (None, 512)               26214912  
                                                                 
 dropout (Dropout)           (None, 512)               0         
                                                                 
 dense_1 (Dense)             (None, 4)                 2052      
                                                                 
Total params: 49,804,676
Trainable params: 49,751,556
Non-trainable params: 53,120
_________________________________________________________________


In [9]:
from tensorflow.keras.callbacks import EarlyStopping

earlystop = EarlyStopping(
    monitor="val_loss", min_delta=0, patience=10, verbose=1, restore_best_weights=True
)

In [10]:
from sklearn.model_selection import train_test_split

x_temp, x_test, y_temp, y_test = train_test_split(
    x_data, y_data, test_size=0.2, random_state=42
)

x_train, x_val, y_train, y_val = train_test_split(
    x_temp, y_temp, test_size=0.25, random_state=42
)

In [11]:
history = model.fit(
    x_train,
    y_train,
    batch_size=32,
    validation_data=(x_val, y_val),
    epochs=50,
    callbacks=earlystop,
)

Epoch 1/50
353/353 [==============================] - 95s 231ms/step - loss: 1.0093 - accuracy: 0.8007 - val_loss: 0.7454 - val_accuracy: 0.7402
Epoch 2/50
353/353 [==============================] - 78s 220ms/step - loss: 0.3635 - accuracy: 0.8908 - val_loss: 51.4447 - val_accuracy: 0.3045
Epoch 3/50
353/353 [==============================] - 76s 216ms/step - loss: 0.3832 - accuracy: 0.8731 - val_loss: 2.3697 - val_accuracy: 0.7264
Epoch 4/50
353/353 [==============================] - 78s 222ms/step - loss: 0.1946 - accuracy: 0.9384 - val_loss: 0.3886 - val_accuracy: 0.8836
Epoch 5/50
353/353 [==============================] - 79s 223ms/step - loss: 0.2922 - accuracy: 0.9126 - val_loss: 0.8887 - val_accuracy: 0.7378
Epoch 6/50
353/353 [==============================] - 79s 224ms/step - loss: 0.1964 - accuracy: 0.9395 - val_loss: 1.9454 - val_accuracy: 0.5887
Epoch 7/50
353/353 [==============================] - 80s 227ms/step - loss: 0.1118 - accuracy: 0.9632 - val_loss: 0.2884 - val_a

In [12]:
plt.figure(figsize=(12, 6))

plt.subplot(1, 2, 1)
plt.plot(history.history["loss"], label="Training Loss")
plt.plot(history.history["val_loss"], label="Validation Loss")
plt.title("Model Loss")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history.history["accuracy"], label="Training Accuracy")
plt.plot(history.history["val_accuracy"], label="Validation Accuracy")
plt.title("Model Accuracy")
plt.xlabel("Epoch")
plt.ylabel("Accuracy")
plt.legend()

In [13]:
test_loss, test_acc = model.evaluate(x_test, y_test)
print(f"Test accuracy: {test_acc}")
print(f"Test loss: {test_loss}")

118/118 [==============================] - 7s 63ms/step - loss: 0.2784 - accuracy: 0.9285
Test accuracy: 0.9285334944725037
Test loss: 0.278375506401062


In [14]:
y_pred_probs = model.predict(x_test)
y_pred = np.argmax(y_pred_probs, axis=1)
y_true = np.argmax(y_test, axis=1)

118/118 [==============================] - 8s 60ms/step


In [15]:
conf_matrix = confusion_matrix(y_true, y_pred)

accuracy = accuracy_score(y_true, y_pred)


precision = precision_score(y_true, y_pred, average="weighted")


recall = recall_score(y_true, y_pred, average="weighted")


f1 = f1_score(y_true, y_pred, average="weighted")

print(f"Accuracy: {accuracy}")

print(f"Precision: {precision}")

print(f"Recall: {recall}")

print(f"F1 Score: {f1}")

Accuracy: 0.9285334750265675
Precision: 0.9322186026877661
Recall: 0.9285334750265675
F1 Score: 0.9283506508951876


In [16]:
plt.figure(figsize=(10, 8))
sns.heatmap(
    conf_matrix,
    annot=True,
    fmt="d",
    cmap="Blues",
    xticklabels=lb.classes_,
    yticklabels=lb.classes_,
)
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.title("Confusion Matrix")
plt.show()

In [17]:
model_directory = "Models"
if not os.path.exists(model_directory):
    os.makedirs(model_directory)
model_path = os.path.join(model_directory, "ResNet50_AUG_02(150x150).h5")
model.save(model_path)
print(f"Model saved to {model_path}")

Model saved to Models\ResNet50_AUG_02(150x150).h5
